### Libs 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import sys
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tabulate import tabulate
from tqdm import tqdm

### Dados

In [2]:
FEATURES_SET = {
    "feature": 1,
    "permission": 2,
    "activity": 3,
    "service_receiver": 3,
    "provider": 3,
    "service": 3,
    "intent": 4,
    "api_call": 5,
    "real_permission": 6,
    "call": 7,
    "url": 8
}


def count_feature_set(lines):
    """
    Count how many features belong to a specific set
    :param lines: features in the text file
    :return:
    """
    features_map = {x: 0 for x in range(1, 9)}
    for l in lines:
        if l != "\n":
            set = l.split("::")[0]
            features_map[FEATURES_SET[set]] += 1
    features = []
    for i in range(1, 9):
        features.append(features_map[i])
    return features


In [3]:
def read(LOAD_DATA=False):
    if LOAD_DATA:
        print("Previous data not loaded. Attempt to read data ...")
        mypath = r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton\Drebin\MetaData\feature_vectors\feature_vectors"
        onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

        print("Reading csv file for ground truth ...")
        ground_truth = np.loadtxt(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton\Drebin\MetaData\sha256_family.csv", delimiter=",", skiprows=1, dtype=str)
        # print ground_truth.shape
        # families = np.unique(ground_truth[:, 1])
        # print families
        # print len(families)

        print("Reading positive and negative texts ...")
        pos = []
        neg = []
        for virus in tqdm(onlyfiles):
            if virus in ground_truth[:, 0]:
                pos.append(virus)
            else:
                #if len(neg) < 5560:
                #if len(neg) < 22240:
                    neg.append(virus)

        print("Extracting features ...")
        x = []
        y = []
        for text_file in tqdm(pos):
            sys.stdin = open("%s/%s" % (mypath, text_file))
            features = sys.stdin.readlines()
            sample = count_feature_set(features)
            x.append(sample)
            y.append(1)

        for text_file in tqdm(neg):
            sys.stdin = open("%s/%s" % (mypath, text_file))
            features = sys.stdin.readlines()
            sample = count_feature_set(features)
            x.append(sample)
            y.append(0)

        print("Data is read successfully:")
        x = np.array(x)
        y = np.array(y)
        print(x.shape, y.shape)

        print("Saving data under data_numpy directory ...")
        np.save(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy", x)
        np.save(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy", y)

        return x, y
    else:
        print("Loading previous data ...")
        x_ = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy")
        y_ = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy")
        print(x_.shape, y_.shape)
        # print x == x_, y == y_
        return x_, y_


def map_family_to_category(families):
    out = {}
    count = 1
    for family in families:
        out[family] = count
        count += 1
    return out


if __name__ == "__main__":
    x, y = read(LOAD_DATA=True)

Previous data not loaded. Attempt to read data ...
Reading csv file for ground truth ...
Reading positive and negative texts ...


100%|██████████| 129013/129013 [00:01<00:00, 70768.04it/s]


Extracting features ...


100%|██████████| 123453/123453 [08:35<00:00, 239.64it/s]


Data is read successfully:
(129013, 8) (129013,)
Saving data under data_numpy directory ...


In [4]:
x_all, y_all = read(LOAD_DATA=False)
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

Loading previous data ...
(129013, 8) (129013,)
(103210, 8) (103210,)
(25803, 8) (25803,)


In [5]:
# # Contagem de classes antes da divisão
# y_all_counts = pd.Series(y_all).value_counts(normalize=True)

# # Divisão dos dados em treinamento e teste
# x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42, stratify=y_all)

# # Contagem de classes após a divisão
# y_train_counts = pd.Series(y_train).value_counts(normalize=True)
# y_test_counts = pd.Series(y_test).value_counts(normalize=True)

# # Verificação se as proporções foram mantidas
# print("Proporções das classes no conjunto original:")
# print(y_all_counts)
# print("\nProporções das classes no conjunto de treinamento:")
# print(y_train_counts)
# print("\nProporções das classes no conjunto de teste:")
# print(y_test_counts)

In [6]:
####### DP = Desvio Padrão; PosBal = Pos Balanceamento
Acurracy = []
Precision = []
Recall = []
F1_score = []
Roc_auc = []
######
AcurracyDP = []
PrecisionDP = []
RecallDP = []
F1_scoreDP = []
Roc_aucDP = []
######
AcurracyBagging = []
PrecisionBagging = []
RecallBagging = []
F1Bagging = []
Roc_aucBagging = []
######
AcurracyBaggingDP = []
PrecisionBaggingDP = []
RecallBaggingDP = []
F1BaggingDP = []
Roc_aucBaggingDP = []
#####
AcurracyPosBal = []
PrecisionPosBal = []
RecallPosBal = []
F1_scorePosBal = []
Roc_aucPosBal = []
######
AcurracyDPPosBal = []
PrecisionDPPosBal = []
RecallDPPosBal = []
F1_scoreDPPosBal = []
Roc_aucDPPosBal = []
######
AcurracyBaggingPosBal = []
PrecisionBaggingPosBal = []
RecallBaggingPosBal = []
F1BaggingPosBal = []
Roc_aucBaggingPosBal = []
######
AcurracyBaggingDPPosBal = []
PrecisionBaggingDPPosBal = []
RecallBaggingDPPosBal = []
F1BaggingDPPosBal = []
Roc_aucBaggingDPPosBal = []

### Modelos Base

In [7]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)

    # Inicializar o modelo de árvore de decisão com Gini Impurity e sem limitação de altura
    modelo_arvore = DecisionTreeClassifier(criterion='gini')

    # Treinar o modelo no conjunto de treino
    modelo_arvore.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_arvore.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:00<00:08,  3.56it/s]

Repetição 1 - Acurácia: 0.98, Precisão: 0.79, Recall: 0.81, F1-score: 0.80, ROC AUC: 0.90


  7%|▋         | 2/30 [00:00<00:07,  3.91it/s]

Repetição 2 - Acurácia: 0.98, Precisão: 0.78, Recall: 0.85, F1-score: 0.81, ROC AUC: 0.92


 10%|█         | 3/30 [00:00<00:06,  3.97it/s]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.77, Recall: 0.83, F1-score: 0.80, ROC AUC: 0.91


 97%|█████████▋| 29/30 [00:06<00:00,  4.20it/s]

Repetição 29 - Acurácia: 0.98, Precisão: 0.77, Recall: 0.79, F1-score: 0.78, ROC AUC: 0.89


100%|██████████| 30/30 [00:07<00:00,  4.15it/s]

Repetição 30 - Acurácia: 0.98, Precisão: 0.77, Recall: 0.83, F1-score: 0.80, ROC AUC: 0.91
Média das Acurácias: 0.98173 ± 0.00069
Média das Precisões: 0.77129 ± 0.01061
Média dos Recalls: 0.81936 ± 0.01282
Média dos F1-scores: 0.79450 ± 0.00767
Média dos ROC AUC: 0.90421 ± 0.00629


In [8]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
    
    #Inicializar o modelo de Naive Bayes com distribuição de Bernoulli
    modelo_naive_bayes = BernoulliNB()

    # Treinar o modelo no conjunto de treino
    modelo_naive_bayes.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_naive_bayes.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  0%|          | 0/30 [00:00<?, ?it/s]

c:\Users\josev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
  3%|▎         | 1/30 [00:00<00:03,  9.13it/s]

Repetição 1 - Acurácia: 0.96, Precisão: 0.00, Recall: 0.00, F1-score: 0.00, ROC AUC: 0.50


c:\Users\josev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\josev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
 10%|█         | 3/30 [00:00<00:01, 13.62it/s]c:\Users\josev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Repetição 2 - Acurácia: 0.96, Precisão: 0.00, Recall: 0.00, F1-score: 0.00, ROC AUC: 0.50
Repetição 3 - Acurácia: 0.96, Precisão: 0.00, Recall: 0.00, F1-score: 0.00, ROC AUC: 0.50
Repetição 4 - Acurácia: 0.96, Precisão: 0.00, Recall: 0.00, F1-score: 0.00, ROC AUC: 0.50


...

Repetição 26 - Acurácia: 0.96, Precisão: 0.00, Recall: 0.00, F1-score: 0.00, ROC AUC: 0.50


c:\Users\josev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
 90%|█████████ | 27/30 [00:01<00:00, 14.84it/s]c:\Users\josev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\josev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
 97%|█████████▋| 

Repetição 27 - Acurácia: 0.96, Precisão: 0.00, Recall: 0.00, F1-score: 0.00, ROC AUC: 0.50
Repetição 28 - Acurácia: 0.96, Precisão: 0.00, Recall: 0.00, F1-score: 0.00, ROC AUC: 0.50
Repetição 29 - Acurácia: 0.96, Precisão: 0.00, Recall: 0.00, F1-score: 0.00, ROC AUC: 0.50


c:\Users\josev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
100%|██████████| 30/30 [00:02<00:00, 14.80it/s]

Repetição 30 - Acurácia: 0.96, Precisão: 0.00, Recall: 0.00, F1-score: 0.00, ROC AUC: 0.50
Média das Acurácias: 0.95690 ± 0.00000
Média das Precisões: 0.00000 ± 0.00000
Média dos Recalls: 0.00000 ± 0.00000
Média dos F1-scores: 0.00000 ± 0.00000
Média dos ROC AUC: 0.50000 ± 0.00000


In [9]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
    
    # Inicializar o modelo MLP
    modelo_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

    # Treinar o modelo no conjunto de treino
    modelo_mlp.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_mlp.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:24<11:58, 24.78s/it]

Repetição 1 - Acurácia: 0.98, Precisão: 0.81, Recall: 0.64, F1-score: 0.72, ROC AUC: 0.82


  7%|▋         | 2/30 [01:01<14:53, 31.90s/it]

Repetição 2 - Acurácia: 0.98, Precisão: 0.82, Recall: 0.59, F1-score: 0.69, ROC AUC: 0.79


 10%|█         | 3/30 [01:38<15:23, 34.19s/it]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.77, Recall: 0.66, F1-score: 0.71, ROC AUC: 0.83


 97%|█████████▋| 29/30 [16:19<00:39, 39.44s/it]

Repetição 29 - Acurácia: 0.98, Precisão: 0.83, Recall: 0.62, F1-score: 0.71, ROC AUC: 0.81


100%|██████████| 30/30 [16:57<00:00, 33.91s/it]

Repetição 30 - Acurácia: 0.98, Precisão: 0.85, Recall: 0.58, F1-score: 0.69, ROC AUC: 0.79
Média das Acurácias: 0.97695 ± 0.00107
Média das Precisões: 0.81735 ± 0.03268
Média dos Recalls: 0.60162 ± 0.02823
Média dos F1-scores: 0.69205 ± 0.01490
Média dos ROC AUC: 0.79774 ± 0.01356


In [10]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
    
    #inicializar o modelo KNN
    modelo_knn = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

    # Treinar o modelo no conjunto de treino
    modelo_knn.fit(np.array(x_train), y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_knn.predict(np.array(x_test))

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:01<00:45,  1.57s/it]

Repetição 1 - Acurácia: 0.98, Precisão: 0.83, Recall: 0.76, F1-score: 0.80, ROC AUC: 0.88


  7%|▋         | 2/30 [00:03<00:42,  1.53s/it]

Repetição 2 - Acurácia: 0.98, Precisão: 0.81, Recall: 0.75, F1-score: 0.78, ROC AUC: 0.87


 10%|█         | 3/30 [00:04<00:41,  1.54s/it]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.82, Recall: 0.74, F1-score: 0.78, ROC AUC: 0.87


 97%|█████████▋| 29/30 [00:41<00:01,  1.44s/it]

Repetição 29 - Acurácia: 0.98, Precisão: 0.81, Recall: 0.75, F1-score: 0.78, ROC AUC: 0.87


100%|██████████| 30/30 [00:43<00:00,  1.45s/it]

Repetição 30 - Acurácia: 0.98, Precisão: 0.81, Recall: 0.76, F1-score: 0.78, ROC AUC: 0.88
Média das Acurácias: 0.98143 ± 0.00070
Média das Precisões: 0.81364 ± 0.01136
Média dos Recalls: 0.73828 ± 0.01337
Média dos F1-scores: 0.77403 ± 0.00894
Média dos ROC AUC: 0.86533 ± 0.00662


In [11]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Acurracy, AcurracyDP)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Precision, PrecisionDP)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Recall, RecallDP)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_score, F1_scoreDP)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_auc, Roc_aucDP)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9817 ± 0.0007,0.7713 ± 0.0106,0.8194 ± 0.0128,0.7945 ± 0.0077,0.9042 ± 0.0063
1,Naive Bayes,0.9569 ± 0.0000,0.0000 ± 0.0000,0.0000 ± 0.0000,0.0000 ± 0.0000,0.5000 ± 0.0000
2,MLP,0.9769 ± 0.0011,0.8174 ± 0.0327,0.6016 ± 0.0282,0.6920 ± 0.0149,0.7977 ± 0.0136
3,KNN,0.9814 ± 0.0007,0.8136 ± 0.0114,0.7383 ± 0.0134,0.7740 ± 0.0089,0.8653 ± 0.0066


### Bagging

In [12]:
# Definir o classificador Decision Tree
modelo_arvore = DecisionTreeClassifier(criterion='gini')

# Definir o classificador de Bagging com Decision Tree como base
bagging_clf = BaggingClassifier(estimator = modelo_arvore, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Decision Tree:", accuracy_bagging)
print("Precisão do modelo Bagging com Decision Tree:", precision_bagging)
print("Recall do modelo Bagging com Decision Tree:", recall_bagging)
print("F1-score do modelo Bagging com Decision Tree:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Decision Tree:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Decision Tree:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)



Acurácia do modelo Bagging com Decision Tree: 0.9890710382513661
Precisão do modelo Bagging com Decision Tree: 0.9036964980544747
Recall do modelo Bagging com Decision Tree: 0.835431654676259
F1-score do modelo Bagging com Decision Tree: 0.8682242990654205
AUC ROC do modelo Bagging com Decision Tree: 0.915711048268833
Matriz de confusão do modelo Bagging com Decision Tree:
[[24592    99]
 [  183   929]]


In [13]:
# Inicializar o modelo de Naive Bayes com distribuição de Bernoulli
modelo_naive_bayes = BernoulliNB()

# Definir o classificador de Bagging com Naive Bayes como base
bagging_clf = BaggingClassifier(estimator=modelo_naive_bayes, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Naive Bayes:", accuracy_bagging)
print("Precisão do modelo Bagging com Naive Bayes:", precision_bagging)
print("Recall do modelo Bagging com Naive Bayes:", recall_bagging)
print("F1-score do modelo Bagging com Naive Bayes:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Naive Bayes:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Naive Bayes:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com Naive Bayes: 0.9569042359415572
Precisão do modelo Bagging com Naive Bayes: 0.0
Recall do modelo Bagging com Naive Bayes: 0.0
F1-score do modelo Bagging com Naive Bayes: 0.0
AUC ROC do modelo Bagging com Naive Bayes: 0.5
Matriz de confusão do modelo Bagging com Naive Bayes:
[[24691     0]
 [ 1112     0]]


c:\Users\josev\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
# Definir o classificador base (MLP)
modelo_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

# Definir o classificador de Bagging com MLP como base
bagging_clf = BaggingClassifier(estimator=modelo_mlp, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com MLP:", accuracy_bagging)
print("Precisão do modelo Bagging com MLP:", precision_bagging)
print("Recall do modelo Bagging com MLP:", recall_bagging)
print("F1-score do modelo Bagging com MLP:", f1_score_bagging)
print("AUC ROC do modelo Bagging com MLP:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com MLP:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com MLP: 0.9795372631089408
Precisão do modelo Bagging com MLP: 0.8526570048309179
Recall do modelo Bagging com MLP: 0.6348920863309353
F1-score do modelo Bagging com MLP: 0.7278350515463917
AUC ROC do modelo Bagging com MLP: 0.8149755073426983
Matriz de confusão do modelo Bagging com MLP:
[[24569   122]
 [  406   706]]


In [15]:
#inicializar o modelo KNN
modelo_knn = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

# Definir o classificador de Bagging com KNN como base
bagging_clf = BaggingClassifier(estimator=modelo_knn, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com KNN:", accuracy_bagging)
print("Precisão do modelo Bagging com KNN:", precision_bagging)
print("Recall do modelo Bagging com KNN:", recall_bagging)
print("F1-score do modelo Bagging com KNN:", f1_score_bagging)
print("AUC ROC do modelo Bagging com KNN:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com KNN:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com KNN: 0.9822888811378522
Precisão do modelo Bagging com KNN: 0.8182701652089407
Recall do modelo Bagging com KNN: 0.7571942446043165
F1-score do modelo Bagging com KNN: 0.7865483418963102
AUC ROC do modelo Bagging com KNN: 0.874810317393487
Matriz de confusão do modelo Bagging com KNN:
[[24504   187]
 [  270   842]]


In [16]:
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']
# Criar um DataFrame do Pandas com os valores das variáveis
dfB = pd.DataFrame({'Modelos Bagging': Modelos, 'Acuracia (DREBIN)': AcurracyBagging,
                    'Precisão (DREBIN)': PrecisionBagging, 'Recall (DREBIN)':RecallBagging, 
                    'F1 Score': F1Bagging, 'AUC ROC': Roc_aucBagging})

print(tabulate(dfB, headers='keys', tablefmt='fancy_grid', floatfmt=".4f"))

╒════╤═══════════════════╤═════════════════════╤═════════════════════╤═══════════════════╤════════════╤═══════════╕
│    │ Modelos Bagging   │   Acuracia (DREBIN) │   Precisão (DREBIN) │   Recall (DREBIN) │   F1 Score │   AUC ROC │
╞════╪═══════════════════╪═════════════════════╪═════════════════════╪═══════════════════╪════════════╪═══════════╡
│  0 │ Decision Tree     │              0.9891 │              0.9037 │            0.8354 │     0.8682 │    0.9157 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  1 │ Naive Bayes       │              0.9569 │              0.0000 │            0.0000 │     0.0000 │    0.5000 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  2 │ MLP               │              0.9795 │              0.8527 │            0.6349 │     0.7278 │    0.8150 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼──

### Balanceamento de dados

In [17]:
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(x_all, y_all)

x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(197524, 8) (197524,)
(49382, 8) (49382,)


In [18]:
# Calcular proporção das classes em y_all
unique_classes, class_counts = np.unique(y_all, return_counts=True)
proporcao_classes_resampled = class_counts / len(y_all)

print("Proporção das classes no conjunto original:")
for classe, proporcao in zip(unique_classes, proporcao_classes_resampled):
    print(f"Classe {classe}: {proporcao:.2f}")

# Calcular proporção das classes em y_resampled
unique_classes, class_counts = np.unique(y_resampled, return_counts=True)
proporcao_classes_resampled = class_counts / len(y_resampled)

print("Proporção das classes no conjunto resampleado:")
for classe, proporcao in zip(unique_classes, proporcao_classes_resampled):
    print(f"Classe {classe}: {proporcao:.2f}")

Proporção das classes no conjunto original:
Classe 0: 0.96
Classe 1: 0.04
Proporção das classes no conjunto resampleado:
Classe 0: 0.50
Classe 1: 0.50


### Modelos Base após o Balanceamento

In [19]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):
    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    # Inicializar o modelo de árvore de decisão com Gini Impurity e sem limitação de altura
    modelo_arvore = DecisionTreeClassifier(criterion='gini')

    # Treinar o modelo no conjunto de treino
    modelo_arvore.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_arvore.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:00<00:15,  1.88it/s]

Repetição 1 - Acurácia: 0.98, Precisão: 0.97, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98


  7%|▋         | 2/30 [00:01<00:14,  1.88it/s]

Repetição 2 - Acurácia: 0.98, Precisão: 0.97, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98


 10%|█         | 3/30 [00:01<00:14,  1.88it/s]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.97, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98


 97%|█████████▋| 29/30 [00:15<00:00,  1.93it/s]

Repetição 29 - Acurácia: 0.98, Precisão: 0.97, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98


100%|██████████| 30/30 [00:15<00:00,  1.90it/s]

Repetição 30 - Acurácia: 0.98, Precisão: 0.98, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98
Média das Acurácias: 0.98119 ± 0.00064
Média das Precisões: 0.97369 ± 0.00089
Média dos Recalls: 0.98911 ± 0.00107
Média dos F1-scores: 0.98134 ± 0.00064
Média dos ROC AUC: 0.98119 ± 0.00064


In [20]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    
    #Inicializar o modelo de Naive Bayes com distribuição de Bernoulli
    modelo_naive_bayes = BernoulliNB()

    # Treinar o modelo no conjunto de treino
    modelo_naive_bayes.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_naive_bayes.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:00<00:04,  7.09it/s]

Repetição 1 - Acurácia: 0.55, Precisão: 0.53, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.55


  7%|▋         | 2/30 [00:00<00:03,  7.20it/s]

Repetição 2 - Acurácia: 0.55, Precisão: 0.53, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.55


 10%|█         | 3/30 [00:00<00:03,  7.41it/s]

...

Repetição 28 - Acurácia: 0.54, Precisão: 0.52, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.54


 97%|█████████▋| 29/30 [00:03<00:00,  7.21it/s]

Repetição 29 - Acurácia: 0.54, Precisão: 0.53, Recall: 0.92, F1-score: 0.67, ROC AUC: 0.54
Repetição 30 - Acurácia: 0.55, Precisão: 0.53, Recall: 0.93, F1-score: 0.67, ROC AUC: 0.55

100%|██████████| 30/30 [00:04<00:00,  7.18it/s]


Média das Acurácias: 0.54515 ± 0.00120
Média das Precisões: 0.52564 ± 0.00070
Média dos Recalls: 0.92549 ± 0.00133
Média dos F1-scores: 0.67048 ± 0.00080
Média dos ROC AUC: 0.54515 ± 0.00120


In [21]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    
    # Inicializar o modelo MLP
    modelo_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

    # Treinar o modelo no conjunto de treino
    modelo_mlp.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_mlp.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [01:34<45:47, 94.73s/it]

Repetição 1 - Acurácia: 0.94, Precisão: 0.95, Recall: 0.92, F1-score: 0.94, ROC AUC: 0.94


  7%|▋         | 2/30 [02:54<40:02, 85.81s/it]

Repetição 2 - Acurácia: 0.94, Precisão: 0.92, Recall: 0.95, F1-score: 0.94, ROC AUC: 0.94


 10%|█         | 3/30 [03:45<31:33, 70.14s/it]

...

Repetição 28 - Acurácia: 0.93, Precisão: 0.91, Recall: 0.96, F1-score: 0.93, ROC AUC: 0.93


 97%|█████████▋| 29/30 [37:05<01:15, 75.65s/it]

Repetição 29 - Acurácia: 0.93, Precisão: 0.91, Recall: 0.95, F1-score: 0.93, ROC AUC: 0.93


100%|██████████| 30/30 [38:15<00:00, 76.51s/it]

Repetição 30 - Acurácia: 0.94, Precisão: 0.95, Recall: 0.93, F1-score: 0.94, ROC AUC: 0.94
Média das Acurácias: 0.93371 ± 0.00250
Média das Precisões: 0.93193 ± 0.01193
Média dos Recalls: 0.93614 ± 0.01473
Média dos F1-scores: 0.93385 ± 0.00282
Média dos ROC AUC: 0.93371 ± 0.00250


In [22]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    
    #inicializar o modelo KNN
    modelo_knn = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

    # Treinar o modelo no conjunto de treino
    modelo_knn.fit(np.array(x_train), y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_knn.predict(np.array(x_test))

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:03<01:48,  3.74s/it]

Repetição 1 - Acurácia: 0.97, Precisão: 0.96, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97


  7%|▋         | 2/30 [00:07<01:45,  3.75s/it]

Repetição 2 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97


 10%|█         | 3/30 [00:11<01:40,  3.72s/it]

...

Repetição 28 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97


 97%|█████████▋| 29/30 [01:41<00:03,  3.49s/it]

Repetição 29 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97


100%|██████████| 30/30 [01:45<00:00,  3.51s/it]

Repetição 30 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97
Média das Acurácias: 0.97072 ± 0.00088
Média das Precisões: 0.95161 ± 0.00158
Média dos Recalls: 0.99188 ± 0.00126
Média dos F1-scores: 0.97133 ± 0.00085
Média dos ROC AUC: 0.97072 ± 0.00088


In [23]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(AcurracyPosBal, AcurracyDPPosBal)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(PrecisionPosBal, PrecisionDPPosBal)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(RecallPosBal, RecallDPPosBal)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_scorePosBal, F1_scoreDPPosBal)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_aucPosBal, Roc_aucDPPosBal)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9812 ± 0.0006,0.9737 ± 0.0009,0.9891 ± 0.0011,0.9813 ± 0.0006,0.9812 ± 0.0006
1,Naive Bayes,0.5451 ± 0.0012,0.5256 ± 0.0007,0.9255 ± 0.0013,0.6705 ± 0.0008,0.5451 ± 0.0012
2,MLP,0.9337 ± 0.0025,0.9319 ± 0.0119,0.9361 ± 0.0147,0.9338 ± 0.0028,0.9337 ± 0.0025
3,KNN,0.9707 ± 0.0009,0.9516 ± 0.0016,0.9919 ± 0.0013,0.9713 ± 0.0009,0.9707 ± 0.0009


### Bagging após o Balanceamento

In [24]:
# Definir o classificador Decision Tree
modelo_arvore = DecisionTreeClassifier(criterion='gini')

# Definir o classificador de Bagging com Decision Tree como base
bagging_clf = BaggingClassifier(estimator = modelo_arvore, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Decision Tree:", accuracy_bagging)
print("Precisão do modelo Bagging com Decision Tree:", precision_bagging)
print("Recall do modelo Bagging com Decision Tree:", recall_bagging)
print("F1-score do modelo Bagging com Decision Tree:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Decision Tree:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Decision Tree:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBaggingPosBal.append(accuracy_bagging)
PrecisionBaggingPosBal.append(precision_bagging)
RecallBaggingPosBal.append(recall_bagging)
F1BaggingPosBal.append(f1_score_bagging)
Roc_aucBaggingPosBal.append(auc_roc_bagging)



Acurácia do modelo Bagging com Decision Tree: 0.9853387874124174
Precisão do modelo Bagging com Decision Tree: 0.9805509884909973
Recall do modelo Bagging com Decision Tree: 0.9903203596452148
F1-score do modelo Bagging com Decision Tree: 0.9854114612718626
AUC ROC do modelo Bagging com Decision Tree: 0.9853387874124175
Matriz de confusão do modelo Bagging com Decision Tree:
[[24206   485]
 [  239 24452]]


In [25]:
# Inicializar o modelo de Naive Bayes com distribuição de Bernoulli
modelo_naive_bayes = BernoulliNB()

# Definir o classificador de Bagging com Naive Bayes como base
bagging_clf = BaggingClassifier(estimator=modelo_naive_bayes, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Naive Bayes:", accuracy_bagging)
print("Precisão do modelo Bagging com Naive Bayes:", precision_bagging)
print("Recall do modelo Bagging com Naive Bayes:", recall_bagging)
print("F1-score do modelo Bagging com Naive Bayes:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Naive Bayes:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Naive Bayes:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBaggingPosBal.append(accuracy_bagging)
PrecisionBaggingPosBal.append(precision_bagging)
RecallBaggingPosBal.append(recall_bagging)
F1BaggingPosBal.append(f1_score_bagging)
Roc_aucBaggingPosBal.append(auc_roc_bagging)


Acurácia do modelo Bagging com Naive Bayes: 0.5430116236685432
Precisão do modelo Bagging com Naive Bayes: 0.5243969676085458
Recall do modelo Bagging com Naive Bayes: 0.9245069053501276
F1-score do modelo Bagging com Naive Bayes: 0.6692074287975843
AUC ROC do modelo Bagging com Naive Bayes: 0.5430116236685433
Matriz de confusão do modelo Bagging com Naive Bayes:
[[ 3988 20703]
 [ 1864 22827]]


In [26]:
# Definir o classificador base (MLP)
modelo_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

# Definir o classificador de Bagging com MLP como base
bagging_clf = BaggingClassifier(estimator=modelo_mlp, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com MLP:", accuracy_bagging)
print("Precisão do modelo Bagging com MLP:", precision_bagging)
print("Recall do modelo Bagging com MLP:", recall_bagging)
print("F1-score do modelo Bagging com MLP:", f1_score_bagging)
print("AUC ROC do modelo Bagging com MLP:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com MLP:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBaggingPosBal.append(accuracy_bagging)
PrecisionBaggingPosBal.append(precision_bagging)
RecallBaggingPosBal.append(recall_bagging)
F1BaggingPosBal.append(f1_score_bagging)
Roc_aucBaggingPosBal.append(auc_roc_bagging)


Acurácia do modelo Bagging com MLP: 0.9437244340042931
Precisão do modelo Bagging com MLP: 0.9392943063352045
Recall do modelo Bagging com MLP: 0.9487667571179782
F1-score do modelo Bagging com MLP: 0.9440067699623219
AUC ROC do modelo Bagging com MLP: 0.943724434004293
Matriz de confusão do modelo Bagging com MLP:
[[23177  1514]
 [ 1265 23426]]


In [27]:
#inicializar o modelo KNN
modelo_knn = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

# Definir o classificador de Bagging com KNN como base
bagging_clf = BaggingClassifier(estimator=modelo_knn, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com KNN:", accuracy_bagging)
print("Precisão do modelo Bagging com KNN:", precision_bagging)
print("Recall do modelo Bagging com KNN:", recall_bagging)
print("F1-score do modelo Bagging com KNN:", f1_score_bagging)
print("AUC ROC do modelo Bagging com KNN:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com KNN:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBaggingPosBal.append(accuracy_bagging)
PrecisionBaggingPosBal.append(precision_bagging)
RecallBaggingPosBal.append(recall_bagging)
F1BaggingPosBal.append(f1_score_bagging)
Roc_aucBaggingPosBal.append(auc_roc_bagging)


Acurácia do modelo Bagging com KNN: 0.9719533433234782
Precisão do modelo Bagging com KNN: 0.9543788505030024
Recall do modelo Bagging com KNN: 0.9912923737394193
F1-score do modelo Bagging com KNN: 0.9724854480799412
AUC ROC do modelo Bagging com KNN: 0.9719533433234783
Matriz de confusão do modelo Bagging com KNN:
[[23521  1170]
 [  215 24476]]


In [28]:
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']
# Criar um DataFrame do Pandas com os valores das variáveis
dfB = pd.DataFrame({'Modelos Bagging': Modelos, 'Acuracia (DREBIN)': AcurracyBaggingPosBal,
                    'Precisão (DREBIN)': PrecisionBaggingPosBal, 'Recall (DREBIN)':RecallBaggingPosBal, 
                    'F1 Score': F1BaggingPosBal, 'AUC ROC': Roc_aucBaggingPosBal})

print(tabulate(dfB, headers='keys', tablefmt='fancy_grid', floatfmt=".4f"))

╒════╤═══════════════════╤═════════════════════╤═════════════════════╤═══════════════════╤════════════╤═══════════╕
│    │ Modelos Bagging   │   Acuracia (DREBIN) │   Precisão (DREBIN) │   Recall (DREBIN) │   F1 Score │   AUC ROC │
╞════╪═══════════════════╪═════════════════════╪═════════════════════╪═══════════════════╪════════════╪═══════════╡
│  0 │ Decision Tree     │              0.9853 │              0.9806 │            0.9903 │     0.9854 │    0.9853 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  1 │ Naive Bayes       │              0.5430 │              0.5244 │            0.9245 │     0.6692 │    0.5430 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  2 │ MLP               │              0.9437 │              0.9393 │            0.9488 │     0.9440 │    0.9437 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼──

### Resultados

In [29]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Acurracy, AcurracyDP)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Precision, PrecisionDP)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Recall, RecallDP)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_score, F1_scoreDP)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_auc, Roc_aucDP)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9817 ± 0.0007,0.7713 ± 0.0106,0.8194 ± 0.0128,0.7945 ± 0.0077,0.9042 ± 0.0063
1,Naive Bayes,0.9569 ± 0.0000,0.0000 ± 0.0000,0.0000 ± 0.0000,0.0000 ± 0.0000,0.5000 ± 0.0000
2,MLP,0.9769 ± 0.0011,0.8174 ± 0.0327,0.6016 ± 0.0282,0.6920 ± 0.0149,0.7977 ± 0.0136
3,KNN,0.9814 ± 0.0007,0.8136 ± 0.0114,0.7383 ± 0.0134,0.7740 ± 0.0089,0.8653 ± 0.0066


In [41]:
import pandas as pd
array = [0,0,0,0,0]
# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f}" for media, dp in zip(AcurracyBagging, array)],
    'Precisão (DREBIN)': [f"{media:.4f}" for media, dp in zip(PrecisionBagging, array)],
    'Recall (DREBIN)': [f"{media:.4f}" for media, dp in zip(RecallBagging, array)],
    'F1 Score': [f"{media:.4f}" for media, dp in zip(F1Bagging, array)],
    'AUC RoC': [f"{media:.4f}" for media, dp in zip(Roc_aucBagging, array)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9891,0.9037,0.8354,0.8682,0.9157
1,Naive Bayes,0.9569,0.0000,0.0000,0.0000,0.5000
2,MLP,0.9795,0.8527,0.6349,0.7278,0.8150
3,KNN,0.9823,0.8183,0.7572,0.7865,0.8748


In [31]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(AcurracyPosBal, AcurracyDPPosBal)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(PrecisionPosBal, PrecisionDPPosBal)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(RecallPosBal, RecallDPPosBal)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_scorePosBal, F1_scoreDPPosBal)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_aucPosBal, Roc_aucDPPosBal)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9812 ± 0.0006,0.9737 ± 0.0009,0.9891 ± 0.0011,0.9813 ± 0.0006,0.9812 ± 0.0006
1,Naive Bayes,0.5451 ± 0.0012,0.5256 ± 0.0007,0.9255 ± 0.0013,0.6705 ± 0.0008,0.5451 ± 0.0012
2,MLP,0.9337 ± 0.0025,0.9319 ± 0.0119,0.9361 ± 0.0147,0.9338 ± 0.0028,0.9337 ± 0.0025
3,KNN,0.9707 ± 0.0009,0.9516 ± 0.0016,0.9919 ± 0.0013,0.9713 ± 0.0009,0.9707 ± 0.0009


In [38]:
import pandas as pd
array = [0,0,0,0,0]
# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f}" for media, dp in zip(AcurracyBaggingPosBal, array)],
    'Precisão (DREBIN)': [f"{media:.4f}" for media, dp in zip(PrecisionBaggingPosBal, array)],
    'Recall (DREBIN)': [f"{media:.4f}" for media, dp in zip(RecallBaggingPosBal, array)],
    'F1 Score': [f"{media:.4f}" for media, dp in zip(F1BaggingPosBal, array)],
    'AUC RoC': [f"{media:.4f}" for media, dp in zip(Roc_aucBaggingPosBal, array)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9853,0.9806,0.9903,0.9854,0.9853
1,Naive Bayes,0.5430,0.5244,0.9245,0.6692,0.5430
2,MLP,0.9437,0.9393,0.9488,0.9440,0.9437
3,KNN,0.9720,0.9544,0.9913,0.9725,0.9720
